In [14]:
import bs4
import requests
import pandas as pd
import json

class Parser(object):

    def __init__(self, time_period, product_name,page):
        self.product_name = product_name
        product_name = product_name.replace(' ','+')
        self.url = f'http://openapi.clearspending.ru/restapi/v3/contracts/search/?daterange={time_period}&productsearch={product_name}&currentstage=EC&page={page}'
        
    def get(self,fname='response'):
        response = requests.get(self.url)
        if response.status_code == 200:
            print('Success!')
        elif response.status_code == 404:
            print('Not Found.')
            raise Exception('ZHOPA')

        data = response.json()

        with open(f'{fname}.json','w',encoding='utf-8') as file:
            json.dump(
                data,
                sort_keys=False,
                indent=4,
                ensure_ascii=False,
                separators=(',', ': '),
                fp=file
                )
        data = None
        with open(f'{fname}.json','r',encoding='utf-8') as json_file:
            data = json.load(json_file)

        #Аттрибуты - дробление на участки данных
        self.contracts = [contract for contract in data['contracts']['data']]
        self.number_of_contracts = len(self.contracts)

        print(f'{self.number_of_contracts} were found')

    def get_compressed_contracts(self):
        self.contracts_data = []
        for contract in self.contracts:
            try:
                self.contracts_data.append(
                    self.__get_compressed_contract(contract)
                    )
            except:
                pass
        return self.contracts_data

    def __get_compressed_contract(self, contract):

        def product_from_list(List, element):
            for product in List:
                if element.lower() in product['name'].lower():
                    return product

        regnum          = contract['regNum']
        customer_name   = contract['customer']['fullName']
        customer_inn    = contract['customer']['inn']
        supplier_name   = contract['suppliers'][0]['organizationName']
        supplier_inn    = contract['suppliers'][0]['inn']
        date            = contract['signDate'].split('T')[0]
    
        product         = product_from_list(contract['products'], self.product_name)
        purchase_object = product['name']
        object_unit     = product['OKEI']['name']
        quantity        = product['quantity']
        price           = product['price']
        cost            = product['sum']
        
        contractUrl     = contract['contractUrl']

        contract_data = {
            'Номер закупки'     :   regnum,
            'Поставщик название':   supplier_name,
            'Поставщик инн'     :   supplier_inn,
            'Заказчик название' :   customer_name,
            'Заказчик инн'      :   customer_inn,
            'Объект закупки'    :   purchase_object,
            'Единица измерения' :   object_unit,
            'Количество'        :   quantity,
            'Цена'              :   price,
            'Стоимость'         :   cost,
            'url'               :   contractUrl,
            'date'              :   date
        }
        return contract_data

    def units(self):
        self.units = []
        for contract in self.contracts_data:
            self.units.append(contract['Единица измерения'])
        self.units = list(set(self.units))
        return self.units

    def find_product(self, product_name):
        for contract in self.contracts:
            print(f"ID: {contract['id']}")
            for ix, product in enumerate(contract['products']):
                if product_name in product['name']:
                   print('{}: - {}'.format(ix+1, product['name']))

    def package_types(self, product_name):
        package_types = 0
        for contract in self.contracts:
            for ix, product in enumerate(contract['products']):
                if product_name in product['name']:
                    try:
                        name = product['name']
                        price = product['price']
                        package = product['OKEI']['name']
                        summa = product['sum']
                        quantity = product['quantity']
                        if package not in package_types:
                            package_types.append(package)
                            print(name, price, package,  quantity, summa)
                    except:
                        pass
        return package_types

In [16]:
for i in range(10):
    p = Parser('01.01.2019-01.01.2020','Формалин',i+1)
    p.get(i+1)
    print(f'{i} out of 10')

Success!
50 were found
0 out of 10
Success!
50 were found
1 out of 10
Success!
50 were found
2 out of 10
Success!
50 were found
3 out of 10
Success!
50 were found
4 out of 10
Success!
50 were found
5 out of 10
Success!
50 were found
6 out of 10
Success!
50 were found
7 out of 10
